<a href="https://colab.research.google.com/github/mirjanaa/Bee-Colony-Optimization/blob/main/genetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [322]:
import numpy as np
import random

In [323]:
class Individual:

  def __init__(self, bounds, obj_function):
    self.code = np.array([random.uniform(bound[0], bound[1]) for bound in bounds])
    self.obj_value = obj_function(self.code)
    self.fitness = self.calc_fitness()

  def __lt__(self, other):
    return self.fitness < other.fitness


  def calc_fitness(self):
    return 1 / (1 + self.obj_value)


In [324]:
bounds = np.array([(-5.12, 5.12), (-5.12, 5.12)])

In [325]:
def rastrigin(x):
    A = 10
    n = len(x)
    return A*n + sum(x_i**2 - A*np.cos(2*np.pi*x_i) for x_i in x)

In [326]:
def selection(population, tournament_size):
  TOURNAMENT_SIZE = tournament_size
  bestFitness = float('-inf')
  index = -1
  for i in range(TOURNAMENT_SIZE):
      randomIndividual = random.randrange(len(population))
      if population[randomIndividual].fitness > bestFitness:
          bestFitness = population[randomIndividual].fitness
          index = randomIndividual
  return index 

In [327]:
def crossover(parent1, parent2, child1, child2):
  
  # breakpoint = random.randrange(len(parent1.code))
    
  # child1.code[:breakpoint] = parent1.code[:breakpoint]
  # child2.code[:breakpoint] = parent2.code[:breakpoint]
  
  # child1.code[breakpoint:] = parent2.code[breakpoint:]
  # child2.code[breakpoint:] = parent1.code[breakpoint:]

  for i in range(len(parent1.code)):
    child1.code[i] = (parent1.code[i] + parent2.code[i]) / 2
    if random.random() < 0.5:
      child2.code[i] = parent1.code[i]
    else:
      child2.code[i] = parent2.code[i]



  print(child1.code)
  print(child2.code)


In [328]:
def mutation(individual):
  MUTATION_PROB = 0.05
  for i in range(len(individual.code)):
      if random.random() < MUTATION_PROB:
          individual.code[i] = random.uniform(bounds[i][0], bounds[i][1])

In [329]:
def genetic_algorithm(population_size, generation_size, tournament_size, obj_function):

  POPULATION_SIZE = population_size
  GENERATION_SIZE = generation_size
  ELITISIM_SIZE = POPULATION_SIZE // 5
  TOURNAMENT_SIZE = tournament_size

  population = [Individual(bounds, obj_function) for _ in range(POPULATION_SIZE)]
  new_population = [Individual(bounds, obj_function) for _ in range(POPULATION_SIZE)]

  for i in range(GENERATION_SIZE):
    population.sort(reverse=True)
    new_population[:ELITISIM_SIZE] = population[:ELITISIM_SIZE]

    for j in range(ELITISIM_SIZE, POPULATION_SIZE, 2):
      index_parent1 = selection(population, TOURNAMENT_SIZE)
      index_parent2 = selection(population, TOURNAMENT_SIZE)

      crossover(population[index_parent1], population[index_parent2], new_population[j], new_population[j+1])

      mutation(new_population[j])
      mutation(new_population[j+1])

      new_population[j].fitness = new_population[j].calc_fitness()
      new_population[j+1].fitness = new_population[j+1].calc_fitness()

    population = new_population

  best_individual = max(population) 

  return best_individual






In [330]:
result = genetic_algorithm(250, 1000, 5, rastrigin)
print("Best solution: ", result.code)

Streaming output truncated to the last 5000 lines.
[-1.0571606  -0.39060171]
[-0.8691084  -1.84947799]
[-0.11184879 -1.04848574]
[-0.53657754 -0.70139496]
[ 0.8593049  -2.34573731]
[-0.9889135  -0.48535035]
[-1.98280011  0.78464197]
[-0.88179985 -0.54133333]
[-1.24422529  0.08361798]
[-1.51998035  0.40310029]
[-1.0571606  0.0215586]
[-0.45342008 -0.68947098]
[-0.98202077 -0.78835695]
[-1.10851312 -0.30394607]
[-1.5994654  -0.39729339]
[1.5006815 0.067717 ]
[ 0.90849885 -0.77955794]
[-0.1502128  -1.96048715]
[-1.626368   -2.65047024]
[-0.76835154 -0.18989877]
[-3.03738458  0.78648709]
[-0.17658818 -1.16628464]
[-1.24422529 -1.16628464]
[-2.06811413 -0.32476723]
[-3.09961999  0.1388225 ]
[ 0.00567149 -1.57749723]
[ 0.00636988 -1.75534266]
[-0.53547372  0.48358953]
[0.17327785 0.08361798]
[ 1.99934604 -0.04058308]
[1.90582793 0.91499193]
[ 0.8593049  -2.34573731]
[ 0.8593049  -2.34573731]
[-2.04727259 -0.40352883]
[-1.0571606  0.0215586]
[-2.02545712  0.22993737]
[-1.98280011  0.78464197]